# Delhi Public School Class 11th Product Setup

This notebook contains all the Supabase SQL queries needed to add a new product for Delhi Public School Class 11th with all the specified combinations and pricing.

## Product Structure:
- **Main Product**: Delhi Public School Class 11th Study Materials
- **Set Types**: NoteBook Set + Bookset, Bookset only
- **Branches**: PCM, PCB, Commerce
- **Optional Subjects**: Physical, CS
- **Total Variants**: 12 combinations with specific pricing

## Step 1: Create School and Category (if not exists)

In [ ]:
-- Create Delhi Public School if it doesn't exist
INSERT INTO schools (name, board, city, state, is_active)
VALUES ('Delhi Public School', 'CBSE', 'Delhi', 'Delhi', true)
ON CONFLICT (name) DO NOTHING;

-- Create books category if it doesn't exist
INSERT INTO categories (name, slug, description, is_active)
VALUES ('Books', 'books', 'Educational books and study materials', true)
ON CONFLICT (slug) DO NOTHING;

## Step 2: Create Main Product

In [ ]:
-- Create the main product
INSERT INTO products (
  sku, 
  title, 
  short_description, 
  description, 
  product_type, 
  base_price, 
  currency, 
  is_active, 
  metadata
) VALUES (
  'DPS-CLASS11-2025',
  'Delhi Public School Class 11th Study Materials',
  'Complete study materials for Class 11th including booksets and notebooks for different streams',
  'Comprehensive study package for Delhi Public School Class 11th students. Available in different combinations for PCM, PCB, and Commerce streams with optional subjects Physical Education and Computer Science.',
  'bookset',
  800.00,
  'INR',
  true,
  '{"school": "Delhi Public School", "class": "11th", "academic_year": "2025"}'::jsonb
);

## Step 3: Link Product to Category and School

In [ ]:
-- Link product to books category
INSERT INTO product_categories (product_id, category_id)
SELECT p.id, c.id
FROM products p, categories c
WHERE p.sku = 'DPS-CLASS11-2025' AND c.slug = 'books';

-- Link product to Delhi Public School for grade 11th
INSERT INTO product_schools (product_id, school_id, grade, mandatory)
SELECT p.id, s.id, '11th', false
FROM products p, schools s
WHERE p.sku = 'DPS-CLASS11-2025' AND s.name = 'Delhi Public School';

## Step 4: Create Product Option Attributes and Values

In [ ]:
-- Create product option attributes and their values
DO $$
DECLARE
    prod_id UUID;
    set_type_attr_id UUID;
    branch_attr_id UUID;
    optional_attr_id UUID;
BEGIN
    -- Get product ID
    SELECT id INTO prod_id FROM products WHERE sku = 'DPS-CLASS11-2025' LIMIT 1;
    
    -- Create Set Type attribute (Position 1)
    INSERT INTO product_option_attributes (product_id, name, position, is_required)
    VALUES (prod_id, 'Set Type', 1, true)
    RETURNING id INTO set_type_attr_id;
    
    -- Create Branch attribute (Position 2)
    INSERT INTO product_option_attributes (product_id, name, position, is_required)
    VALUES (prod_id, 'Branch', 2, true)
    RETURNING id INTO branch_attr_id;
    
    -- Create Optional Subject attribute (Position 3)
    INSERT INTO product_option_attributes (product_id, name, position, is_required)
    VALUES (prod_id, 'Optional Subject', 3, true)
    RETURNING id INTO optional_attr_id;
    
    -- Insert Set Type values
    INSERT INTO product_option_values (attribute_id, value, sort_order) VALUES
    (set_type_attr_id, 'NoteBook Set + Bookset', 1),
    (set_type_attr_id, 'Bookset only', 2);
    
    -- Insert Branch values
    INSERT INTO product_option_values (attribute_id, value, sort_order) VALUES
    (branch_attr_id, 'PCM', 1),
    (branch_attr_id, 'PCB', 2),
    (branch_attr_id, 'Commerce', 3);
    
    -- Insert Optional Subject values
    INSERT INTO product_option_values (attribute_id, value, sort_order) VALUES
    (optional_attr_id, 'Physical', 1),
    (optional_attr_id, 'CS', 2);
    
    RAISE NOTICE 'Product options created successfully for product ID: %', prod_id;
END $$;

## Step 5: Create All Product Variants with Pricing

This creates all 12 variants with the exact pricing specified:

### NoteBook Set + Bookset Variants:
- PCM + Physical: ₹1000
- PCM + CS: ₹1200
- PCB + Physical: ₹900
- PCB + CS: ₹1200
- Commerce + Physical: ₹1000
- Commerce + CS: ₹1000

### Bookset Only Variants:
- PCM + Physical: ₹800
- PCM + CS: ₹1000
- PCB + Physical: ₹700
- PCB + CS: ₹1000
- Commerce + Physical: ₹800
- Commerce + CS: ₹800

Each variant has 25 units in stock.

In [ ]:
-- Create all 12 product variants with specific pricing
DO $$
DECLARE
    prod_id UUID;
    notebook_bookset_id UUID;
    bookset_only_id UUID;
    pcm_id UUID;
    pcb_id UUID;
    commerce_id UUID;
    physical_id UUID;
    cs_id UUID;
BEGIN
    -- Get product ID
    SELECT id INTO prod_id FROM products WHERE sku = 'DPS-CLASS11-2025' LIMIT 1;
    
    -- Get Set Type option value IDs
    SELECT pov.id INTO notebook_bookset_id 
    FROM product_option_values pov
    JOIN product_option_attributes poa ON pov.attribute_id = poa.id
    WHERE poa.product_id = prod_id AND poa.name = 'Set Type' AND pov.value = 'NoteBook Set + Bookset';
    
    SELECT pov.id INTO bookset_only_id 
    FROM product_option_values pov
    JOIN product_option_attributes poa ON pov.attribute_id = poa.id
    WHERE poa.product_id = prod_id AND poa.name = 'Set Type' AND pov.value = 'Bookset only';
    
    -- Get Branch option value IDs
    SELECT pov.id INTO pcm_id 
    FROM product_option_values pov
    JOIN product_option_attributes poa ON pov.attribute_id = poa.id
    WHERE poa.product_id = prod_id AND poa.name = 'Branch' AND pov.value = 'PCM';
    
    SELECT pov.id INTO pcb_id 
    FROM product_option_values pov
    JOIN product_option_attributes poa ON pov.attribute_id = poa.id
    WHERE poa.product_id = prod_id AND poa.name = 'Branch' AND pov.value = 'PCB';
    
    SELECT pov.id INTO commerce_id 
    FROM product_option_values pov
    JOIN product_option_attributes poa ON pov.attribute_id = poa.id
    WHERE poa.product_id = prod_id AND poa.name = 'Branch' AND pov.value = 'Commerce';
    
    -- Get Optional Subject option value IDs
    SELECT pov.id INTO physical_id 
    FROM product_option_values pov
    JOIN product_option_attributes poa ON pov.attribute_id = poa.id
    WHERE poa.product_id = prod_id AND poa.name = 'Optional Subject' AND pov.value = 'Physical';
    
    SELECT pov.id INTO cs_id 
    FROM product_option_values pov
    JOIN product_option_attributes poa ON pov.attribute_id = poa.id
    WHERE poa.product_id = prod_id AND poa.name = 'Optional Subject' AND pov.value = 'CS';
    
    -- Insert all 12 variants with specific pricing
    INSERT INTO product_variants (
        product_id, sku, price, stock, 
        option_value_1, option_value_2, option_value_3, metadata
    ) VALUES
    
    -- NoteBook Set + Bookset variants
    (prod_id, 'DPS-NB-PCM-PHY', 1000.00, 25, notebook_bookset_id, pcm_id, physical_id, 
     '{"set": "NoteBook Set + Bookset", "branch": "PCM", "optional": "Physical"}'::jsonb),
    
    (prod_id, 'DPS-NB-PCM-CS', 1200.00, 25, notebook_bookset_id, pcm_id, cs_id, 
     '{"set": "NoteBook Set + Bookset", "branch": "PCM", "optional": "CS"}'::jsonb),
    
    (prod_id, 'DPS-NB-PCB-PHY', 900.00, 25, notebook_bookset_id, pcb_id, physical_id, 
     '{"set": "NoteBook Set + Bookset", "branch": "PCB", "optional": "Physical"}'::jsonb),
    
    (prod_id, 'DPS-NB-PCB-CS', 1200.00, 25, notebook_bookset_id, pcb_id, cs_id, 
     '{"set": "NoteBook Set + Bookset", "branch": "PCB", "optional": "CS"}'::jsonb),
    
    (prod_id, 'DPS-NB-COM-PHY', 1000.00, 25, notebook_bookset_id, commerce_id, physical_id, 
     '{"set": "NoteBook Set + Bookset", "branch": "Commerce", "optional": "Physical"}'::jsonb),
    
    (prod_id, 'DPS-NB-COM-CS', 1000.00, 25, notebook_bookset_id, commerce_id, cs_id, 
     '{"set": "NoteBook Set + Bookset", "branch": "Commerce", "optional": "CS"}'::jsonb),
    
    -- Bookset only variants
    (prod_id, 'DPS-BO-PCM-PHY', 800.00, 25, bookset_only_id, pcm_id, physical_id, 
     '{"set": "Bookset only", "branch": "PCM", "optional": "Physical"}'::jsonb),
    
    (prod_id, 'DPS-BO-PCM-CS', 1000.00, 25, bookset_only_id, pcm_id, cs_id, 
     '{"set": "Bookset only", "branch": "PCM", "optional": "CS"}'::jsonb),
    
    (prod_id, 'DPS-BO-PCB-PHY', 700.00, 25, bookset_only_id, pcb_id, physical_id, 
     '{"set": "Bookset only", "branch": "PCB", "optional": "Physical"}'::jsonb),
    
    (prod_id, 'DPS-BO-PCB-CS', 1000.00, 25, bookset_only_id, pcb_id, cs_id, 
     '{"set": "Bookset only", "branch": "PCB", "optional": "CS"}'::jsonb),
    
    (prod_id, 'DPS-BO-COM-PHY', 800.00, 25, bookset_only_id, commerce_id, physical_id, 
     '{"set": "Bookset only", "branch": "Commerce", "optional": "Physical"}'::jsonb),
    
    (prod_id, 'DPS-BO-COM-CS', 800.00, 25, bookset_only_id, commerce_id, cs_id, 
     '{"set": "Bookset only", "branch": "Commerce", "optional": "CS"}'::jsonb);
    
    RAISE NOTICE 'All 12 product variants created successfully!';
END $$;

## Step 6: Verify the Setup (Optional)

In [ ]:
-- Verify all variants were created correctly
SELECT 
    pv.sku,
    pv.price,
    pv.stock,
    ov1.value as set_type,
    ov2.value as branch,
    ov3.value as optional_subject
FROM product_variants pv
JOIN products p ON pv.product_id = p.id
JOIN product_option_values ov1 ON pv.option_value_1 = ov1.id
JOIN product_option_values ov2 ON pv.option_value_2 = ov2.id
JOIN product_option_values ov3 ON pv.option_value_3 = ov3.id
WHERE p.sku = 'DPS-CLASS11-2025'
ORDER BY pv.price DESC;

## Setup Complete! ✅

You have successfully created:

- **1 Main Product**: Delhi Public School Class 11th Study Materials
- **3 Option Attributes**: Set Type, Branch, Optional Subject
- **7 Option Values**: 2 set types + 3 branches + 2 optional subjects
- **12 Product Variants** with exact pricing as specified

### Key Features:
- All variants have 25 units in stock
- Proper linking to Delhi Public School and Books category
- Structured metadata for easy filtering and searching
- Unique SKUs for inventory management

### Next Steps:
1. You can now test the product API endpoints
2. Add product images if needed
3. Set up any additional business rules or discounts
4. Configure frontend to display the variants properly

The product is ready for use in your e-commerce system!